# Alumno: Gerardo de Miguel González

### Carga de dataset y visualización

#### Caso Lineal

In [1]:
#::GMG::Cargo los datos del problema lineal (nube de puntos ejemplo)
lin <- read.csv('data/lineal.csv',header = F)

In [2]:
str(lin)

'data.frame':	15 obs. of  3 variables:
 $ V1: num  0.272 0.506 0.526 0.932 0.369 0.524 0.75 0.005 0.641 0.827 ...
 $ V2: num  0.987 0.371 0.9 0.968 0.938 0.196 0.594 0.972 0.926 0.617 ...
 $ V3: int  0 1 0 1 0 1 1 0 0 1 ...


In [3]:
#::GMG::Obtengo los índices de los puntos de una de las clases
ind_l <- which(lin[,3]==0)

In [4]:
#::GMG::Dibujo los puntos con las dos clases
#::nota::
# https://support.rstudio.com/hc/en-us/community/posts/213106048-Figure-size-in-R-Notebook-document
par(mar = c(2,5,1,1))
plot(lin[ind_l,1],lin[ind_l,2],
     type='p',
     xlim=c(0,1),ylim=c(0,1),
     xlab = 'x', ylab = 'y',
     col='blue')
lines(lin[-ind_l,1],lin[-ind_l,2],
      type='p',
      col='red')

### Single-layer perceptron (slp)

**::GMG::** S8-S10, S18-S22 (S01-Intro-FeedForward.pdf), S2-S13 (S03-Lab-Backprop.pdf)

In [9]:
#::GMG::Elijo la función de activación que voy a aplicar (S8-S10)
#       a las neuronas (logistic)
#::nota:: f'(x) = f(x) (1 - f(x)) (S10, S01-Intro-FeedForward.pdf)
sigmoid <- function(z) {
    1/(1 + exp(-z))
}

In [10]:
#::GMG::Construyo la función de FF de la NN (S8, S01-Intro-FeedForward.pdf)
feedforward <- function(x, w) {
  #::nota:: x %*% t(w) (si s hace w %*% t(x) da lo mismo PERO sale un vector fila) 
  return (sigmoid(x %*% t(w)))
}

In [64]:
#::GMG::Construyo la función de aprendizaje backprop
#       con gradient descent (S7-10, S03-Lab-Backprop.pdf)
backprop <- function(y,x, epochs = 10, eta = 0.1) {
    ### Inicializar matrices y listas
    b <- as.matrix(y)
    #::GMG::se añade el bias
    # https://stackoverflow.com/questions/2480650/role-of-bias-in-neural-networks
    a <- cbind (as.matrix(x),1)
    #::GMG::número de pesos a generar
    n <- c(ncol(a),ncol(b))
    #::GMG::Generación aleatoria de pesos iniciales
    w <- matrix(data = runif(prod(n), 
                min = -1, max = 1),
                nrow = n[2], ncol = n[1]
               )
    #::GMG::Ajuste de pesos con gradient descent
    for (j in 1:epochs) {
        ### Propagar hacia delante
        b_hat <- feedforward(a,w)
        ### Actualizar pesos
        error <- b - b_hat
        aux <- error * b_hat * (1 - b_hat)
        delta_w <- eta * (t(aux) %*% a)
        w <- w + delta_w
        ### Error output
        #mean((training.data - predict(training.model))^2)
        sse <- (t(error) %*% error)/2
        #print(sse)
    }
    ### Return values
    colnames (w) <- NULL
    return(list(error = sse, pesos = w))
}

In [53]:
# https://stackoverflow.com/questions/39732029/finding-mean-squared-error
# http://r-statistics.co/Logistic-Regression-With-R.html
# https://github.com/billderose-zz/perceptron/blob/master/perceptron.R
#library(rgl)
# https://rpubs.com/FaiHas/197581
# https://www.kdnuggets.com/2016/08/begineers-guide-neural-networks-r.html/2
# https://sokolj.com/Single-Layer-Perceptron-Implementation/
# http://www.math.univ-toulouse.fr/~xgendre/ens/m2se/M2SE_TP5_en.pdf

# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Perceptron.html#examples-using-sklearn-linear-model-perceptron
# https://scikit-learn.org/stable/modules/neural_networks_supervised.html
# https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/linear_model/perceptron.py#L7
# https://www.rdocumentation.org/packages/neuralnet/versions/1.44.2/source

In [65]:
set.seed(1)

In [69]:
modelo_bp <- backprop(y = lin[,3], x = lin[,-3], epochs = 1000, eta = 0.01)

In [70]:
modelo_bp$error

0.719590507


In [71]:
modelo_bp$pesos

2.700169799,-2.381888626,0.2390269326


In [46]:
library('neuralnet')

In [ ]:
set.seed(1)

In [48]:
modelo_nn <- neuralnet(formula = V3 ~ V2 + V1, data = lin, hidden = 0)

In [50]:
modelo_nn$result.matrix

,1
error,0.416300572748
reached.threshold,0.009295376432
steps,140.000000000000
Intercept.to.V3,0.397804876913
V2.to.V3,-0.826554671030
V1.to.V3,1.271057905633


In [51]:
modelo_nn$weights

0.3978048769
-0.8265546710
1.2710579056


In [52]:
modelo_nn$err.fct

function (x, y) 
{
    1/2 * (y - x)^2
}
<environment: 0x215e930>
attr(,"type")
[1] "sse"

In [58]:
plot(modelo_nn)

In [21]:
lin[,3]

[1] 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1

In [12]:
#::GMG::Obtengo el vector columna (matriz) de entrada de lo que tengo
a_l <- as.matrix(lin[,-3])
dim(a_l)

[1] 15  2

In [13]:
#::GMG::Incluyo el bias 
a_l <- cbind(a_l,rep(1,nrow(a_l)))
dim(a_l)

[1] 15  3

In [15]:
a_l

V1,V2,
0.272,0.987,1
0.506,0.371,1
0.526,0.900,1
0.932,0.968,1
0.369,0.938,1
0.524,0.196,1
0.750,0.594,1
0.005,0.972,1
0.641,0.926,1
0.827,0.617,1


In [16]:
a_l_2 <- cbind (as.matrix(lin[,-3]), 1)
dim (a_l_2)

[1] 15  3

In [17]:
a_l_2

V1,V2,
0.272,0.987,1
0.506,0.371,1
0.526,0.900,1
0.932,0.968,1
0.369,0.938,1
0.524,0.196,1
0.750,0.594,1
0.005,0.972,1
0.641,0.926,1
0.827,0.617,1


In [18]:
#::GMG::Calculo el vector columna (matriz) de salida de lo que quiero
b_l <- as.matrix(lin[,3])
dim(b_l)

[1] 15  1

In [19]:
#::GMG::Construyo las neuronas del single-layer perceptron
#::nota::tres neuronas de entrada y 1 una neurona de salida 
#        sin capas (layers) ocultas
neurons <- c(ncol(a_l),ncol(b_l))
neurons

[1] 3 1

In [ ]:
#::GMG::Recordatorio para hacer el producto escalar de la función sigmoid
# Producto escalar c(1,2,3)%*%c(3,2,1)
#t(b)%*%b
#8

In [20]:
#::GMG::Calculo una matriz de pesos inicial de forma aleatoria
#::nota::uso la función vectorizada prod()
# https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/prod
W <- matrix(data = runif(prod(neurons), min = 0, max = 1),
            nrow = neurons[2], ncol = neurons[1])
dim(W)

[1] 1 3

In [21]:
W

0.2655087,0.3721239,0.5728534


In [24]:
bout <- sigmoid(a_l %*% t(W))

In [27]:
dim(a_l) 
dim(t(W))

[1] 15  3

[1] 3 1

In [31]:
a_l %*% t(W) - t(W %*% t(a_l))

0.000000e+00
0.000000e+00
0.000000e+00
0.000000e+00
2.220446e-16
0.000000e+00
0.000000e+00
0.000000e+00
0.000000e+00
0.000000e+00
-1.110223e-16


In [34]:
dim(bout)
dim (b_l)

[1] 15  1

[1] 15  1

In [36]:
error <- b_l-bout
aux<- error*bout*(1-bout)
Wdelta <- t(aux)%*%a_l

In [37]:
error

-0.7334814
0.3004284
-0.7402796
0.2349581
-0.7349489
0.3132613
0.2703147
-0.7182678
-0.7479354
0.2646323
0.3044511


In [46]:
dim(error)
dim(t(error))

[1] 15  1

[1]  1 15

In [38]:
dim(aux)

[1] 15  1

In [39]:
dim (a_l)

[1] 15  3

In [40]:
dim(Wdelta)

[1] 1 3

In [41]:
dim(W)

[1] 1 3

In [42]:
0.1 * Wdelta

V1,V2,
-0.0009174253,-0.06310155,-0.05555552


In [43]:
Wdelta

V1,V2,
-0.009174253,-0.6310155,-0.5555552


In [45]:
print(a_l)

         V1    V2  
 [1,] 0.272 0.987 1
 [2,] 0.506 0.371 1
 [3,] 0.526 0.900 1
 [4,] 0.932 0.968 1
 [5,] 0.369 0.938 1
 [6,] 0.524 0.196 1
 [7,] 0.750 0.594 1
 [8,] 0.005 0.972 1
 [9,] 0.641 0.926 1
[10,] 0.827 0.617 1
[11,] 0.629 0.232 1
[12,] 0.818 0.295 1
[13,] 0.112 0.318 1
[14,] 0.351 0.813 1
[15,] 0.739 0.706 1


In [ ]:
#::GMG::Llamo a la función backprop() para obtener los pesos óptimos
backprop(b_l, a_l,  epochs = 500, eta = 0.1)